## <strong>0. 시작하기</strong>

이 노트북은 이제 막 데이터 분석에 관심있는 구성원을 위하여 작성되었습니다.

<strong>파라미터의 변경 입력</strong>만으로 <strong>예측 모델의 성능을 개선</strong>해 볼 수 있습니다. 다음의 순서대로 진행해 보세요.

1. TendorFlow 2.x 선택

2. 실행함수 로딩

3. 나만의 모델 학습 및 성능 확인 

4. 제출용 파일 생성 

### <strong>시작하기 전에, GPU를 사용하도록 설정 하셨나요?.</strong>

GPU를 사용하도록 설정해 두면 모델의 학습시간이 단축됩니다.

- 화면 상단의 '<strong>런타임</strong>' 메뉴에서 '<strong>런타임 유형 변경</strong>'을 클릭하세요.
- 런타임 유형은 '<strong>Python 3</strong>'으로, 하드웨어 가속기는 '<strong>GPU</strong>'로 선택하고 '<strong>저장</strong>'을 클릭하세요.



### <strong>이제, 데이터세트를 준비해 주세요.</strong>
- 경연 페이지에서 <strong>train.csv</strong>(학습 및 테스트용)와 <strong>predict_input.csv</strong>(문제용)을 다운로드 하세요.
- 이 노트북 왼쪽 영역에서 '<strong>폴더</strong>' 아이콘을 클릭하세요.
- '<strong>업로드</strong>'를 클릭해서 <strong>train.csv</strong>와 <strong>predict_input.csv</strong> 파일을 업로드하세요.
- 잠시 후, 왼쪽 영역의 파일목록 하단에 업로드 진행 상태가 표시됩니다. 완료가 될때까지 기다려 주세요.


---
## <strong>1. TensorFlow 2.x 선택</strong>




이 노트북에 포함된 코드는 TensorFlow 2.0을 기준으로 작성되었습니다.

Google Colab은 기본적으로 TensorFlow 1.x 환경이기 때문에 TensorFlow 2.x 환경으로 변경하기 위해서는 다음과 같이 별도의 명령을 입력해야 합니다. 

> <strong>다음의 코드를 실행합니다.</strong> (코드 영역을 클릭하면 왼쪽에 실행 버튼이 나타납니다.)

In [0]:
%tensorflow_version 2.x

코드 아래에 실행 결과가 '<strong>TensorFlow 2.x selected.</strong>'로 표시되면 정상적으로 실행된 것입니다.

---
## <strong>2. 실행함수 로딩</strong>

다음의 코드는 여러가지 기능을 하는 함수가 포함되어 있습니다.

- 학습 및 테스트용 데이터세트를 읽어오기

- 모델을 학습, 저장하고 성능을 표시

- 저장된 모델과 문제용 데이터세트를 가져오기 

- 독성을 예측하고 제출용 데이터세트를 생성

코드의 내용을 자세하게 이해할 수 없더라도 일단 실행해 봅시다. 

> <strong>다음의 코드를 실행합니다.</strong> (코드 영역을 클릭하면 왼쪽에 실행 버튼이 나타납니다.)

코드 실행 후 코드의 맨 아래로 화면을 스크롤하면 잠시 후 실행 결과가 순차적으로 나타납니다.

In [0]:
import tensorflow as tf     
import pandas as pd
import numpy as np
import os

from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras import Input
from sklearn.metrics import classification_report,confusion_matrix

def train(
    epoch           = 20,
    batch_size      = 10,
    learning_rate   = 1e-5,
    conv_block_info = [32,3,64,3,64,3],
    fc_block_info   = [128],
    drop_out_rate   = 0.2 
):

    model_save_path = './Train_Result'           
    try:
      if not(os.path.isdir(model_save_path)):
        os.makedirs(os.path.join(model_save_path))

    except OSError as e:
      if e.errno != errno.EEXIST:
        print('학습 결과를 저장할 폴더 생성에 실패하였습니다.')
        raise


    train_data_path = './train.csv'       
    if not os.path.isfile(train_data_path):
        print('학습데이터 파일이 경로에 존재하지 않습니다.')

    dataset         = pd.read_csv(train_data_path)
    print(dataset.head())

    feature         = dataset.iloc[:,1:1025].values.astype(float)     
    label           = dataset['label'].values
    nrow, ncol      = feature.shape

    input_data_size = 1024
    feature         = tf.reshape(feature, (-1, input_data_size, 1, 1))   
    input_shape     = (input_data_size, 1, 1)

    full_dataset    = tf.data.Dataset.from_tensor_slices((feature , label))

    train_size      = int(0.9 * nrow)         
    train_dataset   = full_dataset.take(train_size)
    val_dataset     = full_dataset.skip(train_size)

    train_dataset   = train_dataset.shuffle(train_size).batch(batch_size).repeat()
    val_dataset     = val_dataset.batch(batch_size)

    val_label = []
    for data, label in val_dataset:
      val_label.extend(label.numpy().tolist())

    model = tf.keras.Sequential()
    model.add(Input(shape=input_shape))

    for i in range(0, len(conv_block_info), 2):
      filters_ = int(conv_block_info[i])
      kernel_size_ = int(conv_block_info[i+1])

      model.add(Conv2D(filters=filters_, kernel_size=(kernel_size_, 1), strides=(1, 1), padding='same'))
      model.add(BatchNormalization())
      model.add(Activation('relu'))
      model.add(MaxPool2D((2, 1), padding='same'))

    model.add(Flatten())

    for fc_unit in fc_block_info:
      fc_unit_ = int(fc_unit)

      model.add(Dense(fc_unit_, activation='relu'))
      model.add(Dropout(drop_out_rate))

    model.add(Dense(2, activation='softmax'))
    model.summary()

    adam = tf.keras.optimizers.Adam(learning_rate = learning_rate )
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    model.compile(optimizer = adam, 
                  loss = loss_fn,
                  metrics = ['accuracy'])

    model.fit( train_dataset, epochs = epoch, steps_per_epoch = train_size//batch_size)

    model.save( model_save_path, save_format='tf' )

    val_pred = model.predict( val_dataset )
    val_pred = np.argmax(val_pred,1)

    print('\n-----[Evaluation]-----')
    print(classification_report(val_label, val_pred, digits=4))
    print('\n-----[Confusion Matrix]-----')
    print(confusion_matrix(val_label, val_pred))

def predict():

    model_save_path = './Train_Result'           
    test_data_path = './predict_input.csv'       

    batch_size              = 10
    input_data_size         = 1024

    if not os.path.isfile(test_data_path):
        print('학습데이터 파일이 경로에 존재하지 않습니다.')

    dataset         = pd.read_csv(test_data_path)
    print(dataset.head())

    feature         = dataset.iloc[:,1:1025].values.astype(float)     
    nrow, ncol      = feature.shape

    feature         = tf.reshape( feature , (-1 , input_data_size , 1 , 1) )   

    test_dataset    = tf.data.Dataset.from_tensor_slices( feature )
    test_dataset    = test_dataset.batch( batch_size )

    model = tf.keras.models.load_model( model_save_path ) 

    y_ = model.predict( test_dataset )
    model.summary()

    print(y_)
    y_ = np.argmax(y_,1)

    result = dataset[['SMILES']].copy()
    result['label'] = y_
    result.to_csv('./predict_output.csv', index=False)

train()

위 코드가 정상적으로 실행되면 실행 결과에는 Default 파라미터를 이용한 모델의 학습과정과 성능 평가 결과가 표시됩니다.

- 실행 결과의 제일 마지막 부분에서 1과 0의 <strong>F1 Score</strong>를 확인할 수 있습니다.

- 1은 독성이 없음을, 0은 독성이 있음을 의미합니다. 

- 1(독성 없음)의 F1 Score는 이번 경연에서 순위를 결정짓는 중요한 점수이며 <strong>'1.0'에 가까울수록 높은 성능</strong>을 의미합니다.

---
## <strong>3. 나만의 모델 학습 및 성능 확인</strong>

위 코드의 실행 결과 마지막 부분에서  <strong>[Evaluation]</strong>과 <strong>[Confusion Matrix]</strong>가 표시되면 정상적으로 실행된 것입니다. [Evaluation]의 내용에서 1(독성 없음)의 F1 Score는 얼마였나요?

이제부터 여러분의 생각대로 파라미터를 조정해서 모델을 학습시키고 F1 Score가 어떻게 변하는지 확인하는 방법을 설명하겠습니다. 

우리가 실행시킬 <strong>train()</strong>함수는 총 6개의 파라미터를 지정할 수 있습니다.

- train( <strong>epoch</strong> = 20, <strong>batch_size</strong> = 10, <strong>learning_rate</strong> = 1e-5, <strong>conv_block_info</strong> = [32,3,64,3,64,3], <strong>fc_block_info</strong> = [128], <strong>drop_out_rate</strong> = 0.2 )


각 파라미터의 Default Value와 의미는 다음과 같습니다.
- <strong>epoch</strong> = 20 : 학습용 데이터세트를 이용한 학습의 횟수 = 20번
- <strong>batch_size</strong> = 10 : 한번의 Batch마다 주는 샘플 데이터의 갯수 = 10개
- <strong>learning_rate</strong> = 1e-5 : 모델을 학습시키는 데 사용되는 기울기의 보폭(학습율) = 0.00001
- <strong>conv_block_info</strong> = [32,3,64,3,64,3] : 3개로 구성된 Convolution Layer 각각의 Filter 수 및 Kernal의 크기
- <strong>fc_block_info</strong> = [128] : Fully Connected Layer의 Unit 수 = 128개
- <strong>drop_out_rate</strong> = 0.2 : Overfitting 방지를 위한 Dropout 비율 = 20%


이제 학습의 횟수만 5번으로 변경하여 함수를 실행해 봅시다. 

> <strong>다음의 코드를 실행해 봅시다.</strong>



In [0]:
train( epoch = 5 )

F1 Score이 달라졌나요? 처음보다 올라갔나요?

그럼 epoch와 batch_size를 함께 바꿔보도록 합시다.

다음은 학습 횟수는 5번으로 Batch size는 20개로 변경한 코드입니다.

> <strong>다음의 코드를 실행해 봅시다.</strong>

In [0]:
train( epoch = 5, batch_size = 20 )

F1 Score가 또 변했죠? 

train( )은 위와 같이 원하는 파라미터의 Value를 변경하여 모델을 학습하고 성능을 평가할 수 있는 함수입니다. 

자, 이제 본격적으로 나만의 모델을 위한 작업을 해 봅시다.

> <strong>다음의 코드에서 6가지 파라미터의 Value를 원하는 값으로 변경한 후 실행해 봅시다.</strong>

In [0]:
train( epoch = 20, batch_size = 10, learning_rate = 1e-5, conv_block_info = [32,3,64,3,64,3], fc_block_info = [128], drop_out_rate = 0.2 )


어떻습니까? F1 Score가 좋아지고 있나요? 아니라면 다른 값을 넣어서 다시 실행해 보세요. 원하는 F1 Score를 만들기 위해서는 다양한 Value 값을 이용하여 위 코드를 실행해 봐야 합니다.  

F1 Score를 높이기 위해서 무작위로 수치를 넣는것 보다 CNN(Convolutional Neural Networks)을 조금 공부하고 도전해 보는 것도 괜찮겠죠?

F1 Score가 원하는 만큼 개선이 되었다면 이제 다음의 <strong>'4. 제출용 파일 생성'</strong>으로 넘어가 봅시다.

---
## <strong>4. 제출용 파일 생성</strong>

여기까지 오셨다면 마음에 드는 F1 Score를 찾으신거죠? 하지만 이 F1 Score는 여러분에게 제공된 학습 및 테스트용 데이터세트를 기준으로 표시된 결과입니다. 여러분이 제출하는 파일의 F1 Score는 별도로 계산되며 그 결과는 경연페이지의 리더보드에서 확인할 수 있습니다.

이제 위에서 개발한 모델과 문제용 데이터세트를 이용해서 독성의 유무를 예측하고 제출할 답안용 데이터세트를 만들어 봅시다.

> <strong>아래의 'predict()' 코드를 실행합니다.</strong> 별도의 파라미터 지정은 필요 없습니다.



In [0]:
predict()

- 잠시 후 왼쪽 영역에 '<strong>predict_output.csv</strong>' 파일이 생긴것을 확인할 수 있습니다. 

- 안보이면 '<strong>새로고침</strong>'을 클릭해 보세요.
- 파일에서 마우스 오른쪽 버튼을 클릭한 후 '<strong>다운로드</strong>'를 클릭하세요.
- 원하는 디렉토리에 저장하면 끝.

제출 방법을 참고해서 'predict_output.csv' 파일을 제출해 주세요. 

수고하셨습니다.